# Hidden Markov Models
In this notebook we go through Hidden Markov Models (HHMs) from definition to implementation. We look at the following toy example (taken from Sebastian Thrun's lesson **Happy Grumpy Problem**):

[hgp]: ./assets/rainsun_happygrumpy.png

<center>

![alt text][hgp]

</center>
where we can only observe if person is Happy or Grumpy but not the rain/sun (hidden-state).

Let's define some notation for HMMs

* $\pi$: initial distribution of hidden state
* $a_{ij}$: represents the transition from state $i$ to state $j$
* $A = \left(a_{ij}\right)$: the set of state transition probabilites
* $s_t$: the state at time $t$: 
$$s_t = i \text{ with } i\in\left\{\text{rain, sun}\right\} $$
* $o_t$: the observation at time $t$: 
$$o_t = k \text{ with }k\in\left\{\text{happy, grumpy}\right\} $$
* $b$: state output probability i.e
$$b_i(k) \text{ represents the probability of generating }k \text{ in state }i$$
* $B = \left(b_i(k)\right)$ the set of state output probabilities
* a HMM is often represented by a tuple of $(\pi, A, B)$

The following notebook is organized as following

* Generating rain-sun/happy-grumpy
* Training HMMs using above generated observation to recover original probability

## Generating data
We use the following parameters for our HMM
* $\pi = [0.5, 0.5]$ i.e 
$$P(s_1 = \text{rain}) = P(s_1=\text{sun}) = 0.5$$
* transition matrix is given below
$$
\begin{array}{lcc}
            & s_{t+1}\\
     s_t    & \text{rain} & \text{sun}\\
\text{rain} & 0.6  & 0.4\\
\text{sun}  & 0.2  & 0.8
\end{array}
$$
i.e 
\begin{split}
P(s_{t+1}=\text{rain}\left|\ s_t=\text{rain}\right.) &= 0.6\\
P(s_{t+1}=\text{sun}\left|\ s_t=\text{rain}\right.) &= 0.4\\
P(s_{t+1}=\text{rain}\left|\ s_t=\text{sun}\right.) &= 0.2\\
P(s_{t+1}=\text{sun}\left|\ s_t=\text{sun}\right.) &= 0.8
\end{split}
* state output probabilites is
$$
\begin{array}{lcc}
& o_t \\
     s_t    & \text{grumpy} & \text{happy} \\
\text{rain} & 0.6  & 0.4\\
\text{sun}  & 0.1  & 0.9
\end{array}
$$
i.e
\begin{split}
P(o_t=\text{happy}\left|\ s_t=\text{rain}\right.) &= 0.4\\
P(o_t=\text{grumpy}\left|\ s_t=\text{rain}\right.) &= 0.6\\
P(o_t=\text{happy}\left|\ s_t=\text{sun}\right.) &= 0.9\\
P(o_t=\text{grumpy}\left|\ s_t=\text{sun}\right.) &= 0.1
\end{split}

In [89]:
import numpy as np

# initial probabilities
P0 = np.array([0.5, 0.5])

# transition probabilities
A = np.array([[0.6, 0.4],
              [0.2, 0.8]])

# output probabilities
B = np.array([[0.7, 0.3],
              [0.1, 0.9]])

# map to state to index
hstate_idx_map = {'R' :  0, 'S' :  1}
hidx_state_map = { 0  : 'R', 1  : 'S'}

ostate_idx_map = {'G' : 0, 'H' : 1}

We implement utility functions to generate hidden-states and output-states

In [90]:
def generate_hidden_states(P0, A, T):
    uv = np.random.uniform(size=(T+1))
    st = 0 if uv[0] < P0[0] else 1
    hidden_states = [hidx_state_map[st]]
    for i in range(1, T):
        Pcond = A[st]
        if uv[i] < Pcond[0]:            
            st = 0
        else:            
            st = 1
        # add new hidden-state
        hidden_states.append(hidx_state_map[st])
        
    return hidden_states

def generate_output_states(B, hidden_states):
    uv = np.random.uniform(size=(len(hidden_states)))
    out_states = []
    for i,s in enumerate(hidden_states):
        s_idx = hstate_idx_map[s]
        Pcond = B[s_idx,:]
        
        if uv[i] < Pcond[0]:
            out_states.append('G')
        else:
            out_states.append('H')
    return out_states
            
def generate_hmm_seq(P0, A, B, T):
    hidden_states = generate_hidden_states(P0, A, T)
    out_states = generate_output_states(B, hidden_states)
    return hidden_states, out_states

We generate 10-sequences, each sequences of length $T=20$

In [91]:
hidden_datas   = []
training_datas = []
N = 10
T = 10
for i in range(N):
    h_states, o_states = generate_hmm_seq(P0, A, B, T)
    hidden_datas.append(h_states)
    training_datas.append(o_states)

for i in range(N):
    print ('Hidden-state {}-th:\t{}'.format(i, ''.join(hidden_datas[i])))
    print ('Observation  {}-th:\t{}\n'.format(i, ''.join(training_datas[i])))

Hidden-state 0-th:	SSRRRSSSSS
Observation  0-th:	HHHHHHHHHH

Hidden-state 1-th:	RSSSSRSSSR
Observation  1-th:	HGGHHGHHGG

Hidden-state 2-th:	SSSSSSSSSS
Observation  2-th:	HHHHHHGHHH

Hidden-state 3-th:	RRSSSSSRRS
Observation  3-th:	GGHHHGHHHH

Hidden-state 4-th:	RRSSSSSRRS
Observation  4-th:	GGHHHHHGGH

Hidden-state 5-th:	RSSRSSRSRS
Observation  5-th:	GHHGHHHGHH

Hidden-state 6-th:	RSRRRRRRRR
Observation  6-th:	GHHGGGGHGG

Hidden-state 7-th:	SSSSSSSSSS
Observation  7-th:	HHHHHHHHHH

Hidden-state 8-th:	SSSSSSSRRS
Observation  8-th:	HGHHHHHGHH

Hidden-state 9-th:	RRSSSSRSSS
Observation  9-th:	HGHHGHHHHH



## HMM Decoding/Training
Now, given above observation data, one can ask question: can we recover the parameters for our HMM i.e find 

$$(\pi, A, B)\text{ that maximizes the chance that we see above observation.}$$

We will look at the following algorithm
* [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) for finding the most **likely** sequence of hidden states - called the **Viterbi path**
* [Baum–Welch algorithm](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) for finding the unknown parameters of a HMM

### Viterbi algorithm
Suppose we know the model parameters i.e transition matrix $A$ and emission matrix $B$, and we observe a sequence of output $o_1,\ldots,o_T$, we know want to find the hidden-state $s_1,\ldots,s_T$ that most likely produce the observed sequence $o_t$.

The above problem can be solved by Viterbi algorithm (see [wiki](https://en.wikipedia.org/wiki/Viterbi_algorithm) for more detail). The main idea is to find $s_1,\ldots,s_T$ that maximizes the probability
$$
\mathrm{arg}\max_{s_1,\ldots,s_T}P(o_1,\ldots,o_T,s_1,\ldots,s_T)
$$

Using Markov assumption for HMMs, we can write
$$
P(o_1,\ldots,o_t,s_1,\ldots,s_t) = p(o_t|s_t) \times p(s_t|s_{t-1}) \times P(o_1,\ldots,o_{t-1},s_1,\ldots,s_{t-1})
$$
Look at above recursive form, we can derive the following dynamics programming 

* $V_{1,s} = P(o_1,s_1=s)=P(o_1|s_1=s)\times \pi_s$
* $V_{t,s} = \max_{x\in S} P(o_t|s_t=s)\times P(s_t=s|s_{t-1}=x) \times V_{t-1, x}$

This allow to find the probability of the most probable state sequence $s_1,...,s_t$ that ends at state $s_t=s$.

Let's implement Viterbi algorithm now

In [92]:
def viterbi_decode(observation, P0, A, B):
    hidden_states = []
    S = len(P0)
    V = np.zeros(S)
    # compute log-prob
    logP0 = np.log(P0)
    logA  = np.log(A)
    logB  = np.log(B)
    
    # compute V_{1,s}
    obs = ostate_idx_map[observation[0]]    
    for i in range(S):
        V[i] = logB[i, obs] + logP0[i]   # work in log-space since it might be very small
    
    prev_states = [] # to trace-back
    # dp update V_{t,s}    
    for i in range(1, len(observation)):
        obs_i = ostate_idx_map[observation[i]]
        
        best_states = []
        new_V = np.zeros(S)
        for s in range(S):
            new_v  = float('-inf')
            best_x = None
            for x in range(S):
                v_sx = logA[x, s] + V[x]
                if v_sx > new_v:
                    new_v = v_sx
                    best_x = x
            
            # keep best-x and update V
            best_states.append(best_x)
            new_V[s] = logB[s, obs_i] + new_v
        
        # update V
        V = new_V
        prev_states.append(best_states)
        
    # trace back
    st = np.argmax(V)
    hidden_states.insert(0, hidx_state_map[st])
    for i in range(len(observation)-2,-1,-1):        
        st = prev_states[i][st]
        hidden_states.insert(0, hidx_state_map[st])
    return hidden_states

def decode_error(truth, decode):
    N = len(truth)
    err = 0
    for i in range(N):
        if truth[i] != decode[i]:
            err += 1
    return err/N

In [96]:
for idx in range(N):
    decoded = viterbi_decode(training_datas[idx], P0, A, B)
    print('Test sampe {}-th'.format(idx))
    print('hidden = {}'.format(''.join(hidden_datas[idx])))
    print('decode = {}'.format(''.join(decoded)))
    print('decode-err = {:.2f}\n'.format(decode_error(hidden_datas[idx], decoded)))

Test sampe 0-th
hidden = SSRRRSSSSS
decode = SSSSSSSSSS
decode-err = 0.30

Test sampe 1-th
hidden = RSSSSRSSSR
decode = RRRSSSSSRR
decode-err = 0.40

Test sampe 2-th
hidden = SSSSSSSSSS
decode = SSSSSSSSSS
decode-err = 0.00

Test sampe 3-th
hidden = RRSSSSSRRS
decode = RRSSSSSSSS
decode-err = 0.20

Test sampe 4-th
hidden = RRSSSSSRRS
decode = RRSSSSSRRS
decode-err = 0.00

Test sampe 5-th
hidden = RSSRSSRSRS
decode = RSSSSSSSSS
decode-err = 0.30

Test sampe 6-th
hidden = RSRRRRRRRR
decode = RSSRRRRRRR
decode-err = 0.10

Test sampe 7-th
hidden = SSSSSSSSSS
decode = SSSSSSSSSS
decode-err = 0.00

Test sampe 8-th
hidden = SSSSSSSRRS
decode = SSSSSSSSSS
decode-err = 0.20

Test sampe 9-th
hidden = RRSSSSRSSS
decode = SSSSSSSSSS
decode-err = 0.30



### Baum-Welch algorithm
Now, let's assume that we only observe the output-states without knowing about the parameters $(\pi,A,B)$ and we want to recover $(\pi, A,B)$. This is the goal of [Baum-Welch](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm).

Let's implement it 